In [26]:
import numpy as np
import pandas as pd
import librosa
import os
import glob
import re
import seaborn as sn
import matplotlib.pyplot as plt
import scipy
import time
import collections
import keras
import random
from keras.utils import np_utils
from keras.layers import MaxPooling1D, Conv1D, Conv2D, MaxPooling2D, GlobalAveragePooling2D, LSTM, ELU, Bidirectional, Attention
from keras.layers import Dense, Dropout, Activation, Flatten, CuDNNLSTM
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import tensorflow as tf
import pickle
from keras.utils.vis_utils import plot_model
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
import keras_tuner
from keras import backend as K
import shutil
import os
import tensorflow_addons as tfa
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
from sklearn import metrics

In [27]:
def instrument_code(filename):
    """
    Function that takes in a filename and returns instrument based on naming convention
    """
    # Synth lead borttagen. id = 9
    class_names=['bass', 'brass', 'flute', 'guitar', 
             'keyboard', 'mallet', 'organ', 'reed', 
             'string', 'vocal']
    
    for name in class_names:
        if name in filename:
            return class_names.index(name)
    else:
        return None

In [28]:
with open("CustomDataFull/testdata2000.pkl", 'rb') as f:
    X_test_full = pickle.load(f)

X_test = []
y_test = []

for(key, value) in X_test_full.items():
    X_test.append(value)
    y_test.append(instrument_code(key))
y_test_numpy = np.asarray(y_test)
X_test_numpy = np.asarray(X_test)
print(X_test_numpy.shape)

(4096, 126, 13)


In [32]:
from operator import add
from functools import reduce

model = tf.keras.models.load_model('models/conv2d_87500_126_13_30epochs.h5')

with open('models/conv2d_87500_126_13_30epochs-history', 'rb') as f:
    history = pickle.load(f)

# Score trained model.
scores = model.evaluate(X_test_numpy, y_test_numpy, verbose=1)

y_prediction = model.predict(X_test_numpy)
y_prediction_argmax = np.argmax(y_prediction, axis = 1)

wrong_counter = 0
correct_counter = 0
wrong_file_dict = {}
correct_file_dict = {}

for index, elem in enumerate(y_prediction_argmax):
    if elem != y_test_numpy[index]:
        correct_value = y_test_numpy[index]
        wrong_value = elem
        correct_value_percentage = y_prediction[index][correct_value] * 100
        wrong_value_percentage = y_prediction[index][wrong_value] * 100
        file_name = list(X_test_full.keys())[index]

        wrong_file_dict[file_name] = {"correct_value": correct_value,  "wrong_value" : wrong_value, 
                                       "correct_value_percentage" : correct_value_percentage,
                                       "wrong_value_percentage" : wrong_value_percentage}
    else:
        correct_value = y_test_numpy[index]
        correct_value_percentage = y_prediction[index][correct_value] * 100
        file_name = list(X_test_full.keys())[index]
        correct_file_dict[file_name] = {"correct_value": correct_value, "correct_value_percentage" : correct_value_percentage}

with open('sample_results/missed_samples.txt', 'a') as f:
    for key, value in wrong_file_dict.items():
        f.write('%s:%s\n' % (key, value))

with open('sample_results/correctly_predicted_samples.txt', 'a') as f:
    for key, value in correct_file_dict.items():
        f.write('%s:%s\n' % (key, value))

#for key, value in wrong_file_dict.items():
    #print(key, value)

128/128 [==============================] - 0s 2ms/step
